# Camera intrinsics, screen coordinates and 3D data.

At first, we install some dependencies.

Installing `face_api_dataset` from the github it it's not already installed.

In [ ]:
%%bash

(python -c "import face_api_dataset")
if [ $? -eq 0 ]
then
    echo "face_api_dataset is already installed"
else
    echo "installing face_api_dataset from github"
    pip install git+https://github.com/Synthesis-AI-Dev/face_api_dataset.git
fi

Installing `git lfs` from the github it it's not already installed.

In [ ]:
%%bash

(git lfs install > /dev/null)
if [ $? -eq 0 ]
then
    echo "git lfs is already installed"
else
    echo "installing git lfs"
    case "$(uname -s)" in

       Darwin)
         brew install git-lfs
         ;;

       Linux)
         curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
         apt-get install git-lfs
         ;;

       *)
         echo "This cell can't install git lfs your OS. Please take care of it yourself."
         ;;
    esac
fi

Pulling the test dataset for the example

In [ ]:
%%bash

if [ -d "../test_dataset" ]
then
    echo "test_dataset is already present"
else
    echo "downloading test_dataset"
    git clone https://github.com/Synthesis-AI-Dev/face_api_dataset
    mv face_api_dataset/test_dataset ..
    rm -rf face_api_dataset
fi

Finally we add some imports for the visualisation.

In [ ]:
import cv2
%matplotlib inline

In [ ]:
import numpy as np

We use `FaceApiDataset` class to access synthesis datasets.


In [ ]:
from face_api_dataset import FaceApiDataset, Modality

**Warning!** Some of modalities requires additional libraries to be installed:
`SEGMENTS` and `RGB` modalities use `opencv-python` library,
 while `DEPTH`, `ALPHA` and `NORMALS` modalities
 use `tiffile`  and `imagecodecs` libraries for effective work with floating point tiff files.
 If dataset with these modalities will be created without corresponding libraries present, an `ImportError` is raised.


In [ ]:
data_root = "../test_dataset"
dataset = FaceApiDataset(data_root,
                        modalities=[Modality.RGB, Modality.CAM_INTRINSICS, Modality.LANDMARKS_3D_IBUG68,
                                    Modality.LANDMARKS_IBUG68, Modality.DEPTH
                                   ])

In [ ]:
len(dataset)

There are 13 items in the test dataset. Let's explore them closer.

In [ ]:
item = dataset[0]
item2 = dataset[1]

Each item is a dict with different modalities as keys.

In [ ]:
print(item.keys())

In this notebook we explore camera intrinsics and transistion between 3D and the screen coordinate systems.

In [ ]:
intrinsics = item[Modality.CAM_INTRINSICS]
intrinsics

The camera intrinsics are in OpenCV format:
$$\left [ \begin{matrix} f_x & 0 & c_x \\ 0 & f_y & c_y \\ 0 & 0 & 1 \end{matrix} \right ]$$,
where $f_x$ and $f_y$ are focal distances and $c_x$ and $c_y$ is an optical center
(2D coordinates of the point, camera is pointing at on the image).

Note, that units of distance in 3D are meters and units in 2D are pixels, the conversion coefficient is already included in $f_x$.

$c_x$ and $c_y$ are usually equal to the half of the image resolution as camera is normally pointing to the center of the image.

In [ ]:
np.array(item[Modality.RGB].shape[:2]) / 2

Now we can convert between camera and screen coordinate systems. Let's see how it is done, using ibug68 landmarks.

In [ ]:
landmark_0 = item[Modality.LANDMARKS_3D_IBUG68][0]
landmark_0

At first, we need to check that landmark is in front of the camera (z coordinate is negative):

In [ ]:
landmark_0[2] < 0


Now we can convert to screen coordinate system, using intrinsic matrix. However, our 3D camera coordinate system uses OpenGL axis (x right, y top, z towards the camera), and OpenCV system uses a different [one](https://docs.opencv.org/4.5.3/d9/d0c/group__calib3d.html) (x right, y bottom, z from the camera).
Thus before we apply the conversion, we need to convert between this coordinate systems, otherwise screen coordinates will be mirrored vertically.

In [ ]:
def opengl_to_opencv(x):
    return np.array(x) * [1, -1, -1]

To compute landmark in screen coordinates we need to multiply 3D coordinates by the intrinsic matrix.

In [ ]:
def cam_to_hom_screen(intrinsics, x):
    return np.tensordot(opengl_to_opencv(x), intrinsics, axes=(-1, 1))

In [ ]:
cam_to_hom_screen(intrinsics, landmark_0)

We get the result, but it is in homogeneous coordinates. We need to convert it to euclidian.

In [ ]:
def from_homogeneous_2D(xs):
    xs_array = np.array(xs)
    assert (xs_array.shape[-1] == 3)

    slice_but_last = [slice(None)] * (xs_array.ndim - 1) + [slice(None, -1)]
    slice_but_last = tuple(slice_but_last)

    slice_last = [slice(None)] * (xs_array.ndim - 1) + [slice(-1, None)]
    slice_last = tuple(slice_last)
    return xs_array[slice_but_last] / xs_array[slice_last]

In [ ]:
def cam_to_screen(intrinsics, x):
    return from_homogeneous_2D(cam_to_hom_screen(intrinsics, x))


In [ ]:
cam_to_screen(intrinsics, landmark_0)

To check ourselves, let's compare with 2D location of this landmark we get from the info.json:

In [ ]:
item[Modality.LANDMARKS_IBUG68]['0']

Now let's have a look, how landmarks look in 3D.

We use ploply to visualise 3D data.

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

import pandas as pd

In [ ]:
x, y, z =  np.array(list(item[Modality.LANDMARKS_3D_IBUG68].values()), dtype=np.float64).transpose()
lid = list(item[Modality.LANDMARKS_3D_IBUG68].keys())

Let's visualize landmarks with labels:

In [ ]:
fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z, text=lid, mode='markers+text', marker={"size": 2})])
fig.update_layout(scene_camera={"eye": {"x": 0., "y": 0., "z": 2},
                                "up": {"x": 0, "y": 1, "z": 0}}, title="iBUG68 3D labeled")
fig.show()

And without them for more clarity:

In [ ]:
fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z, mode='markers+text', marker={"size": 2})])
fig.update_layout(scene_camera={"eye": {"x": 0., "y": 0., "z": 2},
                                "up": {"x": 0, "y": 1, "z": 0}}, title="iBUG68 3D")
fig.show()

Now we can project all the landmarks to the screen and check that position is correct on the image as well:

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def landmark_show(img, landmarks, radius=2, labels=True):
    l_img = np.copy(img)
    for x, y in landmarks:
        int_p = (int(x), int(y))
        cv2.circle(l_img, int_p, radius=radius, color=(255, 0, 0), thickness=cv2.FILLED)
    plt.imshow(l_img)

In [ ]:
landmarks_2D = cam_to_screen(intrinsics, np.array(list(item[Modality.LANDMARKS_3D_IBUG68].values()), dtype=np.float64))

In [ ]:
plt.figure(figsize=(10,10))
landmark_show(item[Modality.RGB], landmarks_2D)

From intrinsic transform we can compute reverse intrinsic transform, which is needed to map points from the image to 3D space.

In [ ]:
rev_intrinsics = np.linalg.inv(intrinsics)

Screen coordinates are equivealent to the pixel position:

In [ ]:
shape = item[Modality.RGB].shape
screen_xs, screen_ys = np.meshgrid(np.linspace(0, shape[1] - 1, shape[1]),
                                   np.linspace(0, shape[0] - 1, shape[0]))
depth = item[Modality.DEPTH]
color = item[Modality.RGB]

To apply reverse projection we need to homogenize the coordinates and multiply them by reversed intrinsic matrix:

In [ ]:
xs, ys, zs = np.moveaxis(np.tensordot(np.stack([screen_xs * depth, screen_ys * depth, depth]), rev_intrinsics, axes=(0, 1)), -1, 0) * np.array([1,-1,-1]).reshape(-1, 1, 1)

Now we can show point cloud of the head.

In [ ]:
fig = go.Figure(data=[go.Scatter3d(x=xs[zs<0], y=ys[zs<0], z=zs[zs<0],
                                   mode='markers+text', marker={"size": 1, "color": color[zs<0]})])
fig.update_layout(scene_camera={"eye": {"x": 0., "y": 0., "z": 2},
                                "up": {"x": 0, "y": 1, "z": 0}}, title="Head 3D reconstruction")
fig.show()


To check that everything is correct let's display landmarks at the same 3D plot as well:

In [ ]:
x, y, z =  np.array(list(item[Modality.LANDMARKS_3D_IBUG68].values()), dtype=np.float64).transpose()

fig = go.Figure(data=[go.Scatter3d(x=xs[zs<0], y=ys[zs<0], z=zs[zs<0],
                                   mode='markers+text', marker={"size": 1, "color": color[zs<0]}),
                      go.Scatter3d(x=x, y=y, z=z, mode='markers+text', marker={"size": 3})])
fig.update_layout(scene_camera={"eye": {"x": 0., "y": 0., "z": 2},
                                "up": {"x": 0, "y": 1, "z": 0}}, title="Head 3D reconstruction with landmarks",
                  showlegend=False)
fig.show()
